In [ ]:
import numpy as np
import pandas as pd

In [ ]:
mood_data = pd.read_csv('data_moods .csv')
data = pd.read_csv('data.csv')

In [ ]:
mood_data

In [ ]:
data

In [ ]:
features = ['acousticness','danceability', 'energy','instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo', 'duration_ms', 'key']


# Clustering Emotions Data.

In [ ]:
numerical_data = mood_data[features]

In [ ]:
from sklearn.preprocessing import StandardScaler

song_moods_scaler = StandardScaler()
scale_data = song_moods_scaler.fit_transform(numerical_data)

In [ ]:
from sklearn.cluster import KMeans

mood_cluster = KMeans(n_clusters = 5)
mood_cluster.fit(scale_data)

In [ ]:
from sklearn.decomposition import PCA

mood_reducer = PCA(n_components = 2)
reduce_data = mood_reducer.fit_transform(scale_data)

In [ ]:
cluster_labels = mood_cluster.labels_
mood_data['cluster'] = cluster_labels

In [ ]:
new_mood_data = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduce_data)

new_mood_data['name'] = mood_data['name']
new_mood_data['mood'] = mood_data['mood']
new_mood_data['cluster'] = mood_data['cluster']

In [ ]:
new_mood_data

In [ ]:
import plotly.express as px

graph = px.scatter(new_mood_data, x = 'x-axis', y = 'y-axis', color = 'cluster', hover_data = ['x-axis', 'y-axis', 'name'])

graph.show()

# Clustering of songs with names

In [ ]:
# taking the numeric data

numeric_song_data = data[features]
numeric_song_data

In [ ]:
# scaling the data
from sklearn.preprocessing import StandardScaler

song_scaler = StandardScaler()
scaled_song_data = song_scaler.fit_transform(numeric_song_data)

In [ ]:
# clustering the scaled data.
from sklearn.cluster import KMeans

song_cluster = KMeans(n_clusters = 20)
song_cluster.fit(scaled_song_data)

In [ ]:
# labeling song clusters
song_cluster_labels = song_cluster.labels_

# addinng cluster lables to the datset
data['clusters'] = song_cluster_labels


In [ ]:
# reducing the dimensionality for graph plotting.
# as data has linear relation instead of TSNE we use PCA

from sklearn.decomposition import PCA

song_reducer = PCA(n_components = 2)
reduced_song_data = song_reducer.fit_transform(scaled_song_data)

In [ ]:
new_song_dataset = pd.DataFrame(columns = ['x-axis', 'y-axis'], data = reduced_song_data)
new_song_dataset['name'] = data['name']
new_song_dataset['clusters'] = data['clusters']

In [ ]:
import plotly.express as px

song_graph = px.scatter(new_song_dataset, x = 'x-axis', y = 'y-axis', color = 'clusters', hover_data = ['x-axis', 'y-axis', 'clusters'])
song_graph.show()

In [ ]:
distinct = mood_data['mood'].unique()
distinct

In [ ]:
Grouped_data = mood_data.groupby('mood')

In [ ]:
mean_values = Grouped_data[features].mean()

In [ ]:
moods = ['Calm', 'Energetic', 'Happy', 'Sad']
mean_values['mood'] = moods

In [ ]:
mean_values

# Recommending songs Based on Mood

## Song name as Input

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def MoodBasedRecommender1(song_name):
    song_features = data.loc[data['name'] == song_name, features].iloc[0]
    best_mood_match = None
    best_mood_similarity = -1
    #Finding the mood of the song
    for index,row in mean_values.iterrows():
        mood_features = row[features]
        similarity = cosine_similarity([song_features], [mood_features])[0][0]
        if similarity > best_mood_similarity:
            best_mood_match = row['mood']
            best_mood_similarity = similarity
    song_mood = best_mood_match
    #finding song cluster
    song_cluster = data.loc[data['name'] == song_name, 'clusters'].iloc[0]
    
    #Filtering similar cluster songs
    similar_cluster_songs = data.loc[data['clusters'] == song_cluster]
    
    best_songs = {}
    mood_features = mean_values.loc[mean_values['mood'] == song_mood, features].iloc[0]
    best_song_match = None
    best_song_similarity = -1
    
    for index,row in similar_cluster_songs.iterrows():
        song_features = row[features]
        song_similarity = cosine_similarity([song_features],[mood_features])[0][0]
        if song_similarity > best_song_similarity:
            best_song_match = row['name']
            best_song_similarity = song_similarity
            best_songs[best_song_match] = best_song_similarity
    return best_songs

In [ ]:
# Calling the recommending function
best_songs = MoodBasedRecommender1("Clancy Lowered the Boom")
# sorting the best similar songs
sorted_best_songs = dict(sorted(best_songs.items(), key = lambda x: x[1]))
# Printing the recommending songs
best_songs_list = list(sorted_best_songs.keys())
for i in range(len(best_songs_list)):
    if i <= 4 :
        print(best_songs_list[i])
    else:
        break

## mood as input

In [ ]:
def MoodBasedRecommender2(mood):
    best_songs = {}
    mood_features = mean_values.loc[mean_values['mood'] == mood, features].iloc[0]
    best_song_match = None
    best_song_similarity = -1
    
    for index,row in data.iterrows():
        song_features = row[features]
        song_similarity = cosine_similarity([song_features],[mood_features])[0][0]
        if song_similarity > best_song_similarity:
            best_song_match = row['name']
            best_song_similarity = song_similarity
            best_songs[best_song_match] = best_song_similarity
    return best_songs

In [ ]:
# Calling the recommending function
best_songs = MoodBasedRecommender2("Happy")
# sorting the best similar songs
sorted_best_songs = dict(sorted(best_songs.items(), key = lambda x: x[1]))
# Printing the recommending songs
best_songs_list = list(sorted_best_songs.keys())
for i in range(len(best_songs_list)):
    if i <= 4 :
        print(best_songs_list[i])
    else:
        break